# Video Feature Extraction using Mediapipe Pose

This notebook extracts pose keypoints (features) from videos located in specified `train_data` and `test_data` folders using Mediapipe Pose.

**Workflow:**
1.  **Setup:** Install compatible librarie using settings.
2.  **Preprocessing Functions:** Define functions to extract keypoints from a video and process folders.
3.  **Feature Extraction:** Run the extraction process on all videos in the specified train and test directories, saving the results as `.pkl` files.

## Cell 1: Setup Environment - Install Compatible Libraries, Mount Drive & Import

Installs necessary and compatible library versions (pinning NumPy), mounts Google Drive, and imports required modules. 

**IMPORTANT:** After running this cell the first time, **RESTART THE RUNTIME** using the menu (`Runtime` -> `Restart Runtime`), then **RUN THIS CELL AGAIN** before proceeding to Cell 2.

In [ ]:
# Cell 1: Setup Environment - Install Compatible Libraries, Mount Drive & Import

# Uninstall potentially conflicting libraries first for a cleaner state
print("Uninstalling potentially conflicting libraries...")
!pip uninstall numpy torch torchvision torchaudio mediapipe opencv-python -y

print("\nInstalling specific NumPy version...")
!pip install --no-cache-dir --force-reinstall numpy

print("\nInstalling other libraries without dependency changes...")
!pip install --no-cache-dir --no-dependencies torch torchvision torchaudio mediapipe opencv-python

print("\nLibrary installation attempt finished.")
print("------------------------------------------")


# --- Code below will run AFTER restarting and running this cell again ---

# -*- coding: utf-8 -*-
import os

# Import necessary libraries (now that installations should be correct)
print("\nImporting libraries...")
import cv2
import numpy as np
import mediapipe as mp
import time
import pickle
import glob

print("Required libraries for feature extraction imported.")

# Check crucial versions
try:
    print(f"\nUsing NumPy version: {np.__version__}")
    print(f"Using Mediapipe version: {mp.__version__}")
    # Check if the NumPy version is the one we intended
    if np.__version__.startswith('1.26'):
        print("\nNumPy version 1.26.x successfully loaded.")
    else:
        print(f"\nWARNING: NumPy version is {np.__version__}, not 1.26.x! Dependency conflict might still exist or restart was missed.")
except NameError:
    print("\nLibraries not fully imported yet (expected on first run before restart). Please follow restart instructions.")

## Cell 2: Configuration - Define Paths and Hyperparameters

Set up all the configuration variables: paths to your `train_data` and `test_data` folders, the output directory for features, class names, and feature extraction settings (frame skip, sequence length).

In [ ]:
# --- User Defined Paths ---
# !! IMPORTANT: Update BASE_DRIVE_PATH if your 'dataset' folder is located elsewhere in Drive !!
BASE_DRIVE_PATH = '/content/drive/MyDrive/dataset' # Main folder in your Drive containing train_data and test_data

#  Noted that, I have change the flow abit. This train and test data will be combined together after extract the feature and keep in the same directory.
# The train and test will be splited during training phase
TRAIN_DATA_DIR = os.path.join(BASE_DRIVE_PATH, 'train_data')
TEST_DATA_DIR = os.path.join(BASE_DRIVE_PATH, 'test_data')

# Class folder names (should be consistent within train_data and test_data)
# !! IMPORTANT: Update CLASS_FOLDERS if your class names are different !!
CLASS_FOLDERS = ["backward_fall", "forward_fall", "side_fall", "non_fall"]

# Output directories (relative to BASE_DRIVE_PATH)
FEATURES_OUTPUT_DIR = os.path.join(BASE_DRIVE_PATH, 'processed_features') # Combined features folder

# --- Preprocessing Settings ---
FRAME_SKIP = 3  # Extract every Nth frame (e.g., 3 means ~10fps from 30fps, or ~20fps from 60fps)
SEQUENCE_LENGTH = 30 # Fixed number of frames (keypoint sequences) per video sample
INPUT_SIZE = 33 * 3  # 33 landmarks * 3 coordinates (x, y, visibility) - Needed for padding

# --- Create output directories if they don't exist ---
print(f"Creating base feature output directory: {FEATURES_OUTPUT_DIR}")
os.makedirs(FEATURES_OUTPUT_DIR, exist_ok=True)
# Create class subdirectories within features directory if they don't exist
print("Creating class subdirectories for features...")
for class_name in CLASS_FOLDERS:
    os.makedirs(os.path.join(FEATURES_OUTPUT_DIR, class_name), exist_ok=True)

# --- Define class to index mapping (needed for process_data_folder, though labels aren't used later) ---
label_map = {name: i for i, name in enumerate(CLASS_FOLDERS)}

print("Configuration loaded:")
print(f"  Base Path: {BASE_DRIVE_PATH}")
print(f"  Train Data Path: {TRAIN_DATA_DIR}")
print(f"  Test Data Path: {TEST_DATA_DIR}")
print(f"  Feature Output Path: {FEATURES_OUTPUT_DIR}")
print(f"  Classes: {CLASS_FOLDERS}")
print(f"  Frame Skip: {FRAME_SKIP}")
print(f"  Sequence Length: {SEQUENCE_LENGTH}")

## Cell 3: Preprocessing Function Definitions

Initializes the Mediapipe Pose model and defines the helper functions:
*   `extract_keypoints_from_video`: Extracts pose landmarks from a video file, handling padding/truncation.
*   `process_data_folder`: Iterates through class folders, processes each video using the above function, and saves the extracted keypoint sequence as a `.pkl` file.

In [ ]:
# Initialize Mediapipe Pose
print("Initializing Mediapipe Pose...")
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, # Process video stream
                    model_complexity=1,      # 0, 1, 2 (higher is more accurate but slower)
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

# mp_drawing = mp.solutions.drawing_utils # Not needed for feature extraction only

def extract_keypoints_from_video(video_path, frame_skip, sequence_length):
    """
    Extracts Mediapipe keypoints from a video file. Handles sequence padding/truncation.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return None

    frame_count = 0
    keypoints_sequence = []

    while cap.isOpened():
        success, frame = cap.read()
        if not success: break # End of video

        if frame_count % frame_skip == 0:
            try:
                # Convert the BGR image to RGB.
                image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # To improve performance, optionally mark the image as not writeable to
                # pass by reference.
                image_rgb.flags.writeable = False
                # Process the image and find pose landmarks.
                results = pose.process(image_rgb)
                # image_rgb.flags.writeable = True # No drawing needed

                if results.pose_landmarks:
                    frame_keypoints = []
                    for landmark in results.pose_landmarks.landmark:
                        # Store x, y, visibility
                        frame_keypoints.extend([landmark.x, landmark.y, landmark.visibility])
                    keypoints_sequence.append(frame_keypoints)
                else:
                    # Pad with zeros if no pose detected in a frame
                    keypoints_sequence.append(np.zeros(INPUT_SIZE).tolist())
            except Exception as e:
                print(f"Error processing frame {frame_count} in {video_path}: {e}")
                # Pad with zeros on error
                keypoints_sequence.append(np.zeros(INPUT_SIZE).tolist())

        frame_count += 1
    cap.release()

    num_extracted_frames = len(keypoints_sequence)
    if num_extracted_frames == 0:
        print(f"Warning: No keypoints extracted from {video_path}")
        return None # Return None if video was empty or unreadable

    # Pad or truncate the sequence to the desired sequence_length
    final_sequence = np.zeros((sequence_length, INPUT_SIZE), dtype=np.float32)

    if num_extracted_frames >= sequence_length:
        # Truncate if too long
        final_sequence = np.array(keypoints_sequence[:sequence_length], dtype=np.float32)
    else:
        # Pad with zeros if too short
        sequence_array = np.array(keypoints_sequence, dtype=np.float32)
        final_sequence[:num_extracted_frames] = sequence_array

    return final_sequence


def process_data_folder(data_directory, class_folders, label_map, feature_output_dir):
    """Processes videos in a given directory (train or test) and saves features."""
    processed_files_count = 0
    skipped_files_count = 0
    failed_files_count = 0
    print(f"\nStarting processing for data source: {data_directory}")

    for class_name in class_folders:
        class_label = label_map[class_name]
        video_folder_abs = os.path.join(data_directory, class_name)
        # Features are saved in a common structure under FEATURES_OUTPUT_DIR
        feature_class_dir = os.path.join(feature_output_dir, class_name)

        print(f"\n  Processing class folder: {class_name}")
        print(f"    Video source folder: {video_folder_abs}")
        print(f"    Feature output folder: {feature_class_dir}")

        # Find video files (e.g., .mp4, .avi)
        video_files = []
        # Add common video extensions
        for ext in ('*.mp4', '*.avi', '*.mov', '*.wmv', '*.mkv', '*.flv'):
             video_files.extend(glob.glob(os.path.join(video_folder_abs, ext)))
             video_files.extend(glob.glob(os.path.join(video_folder_abs, ext.upper()))) # Also check uppercase extensions

        video_files = sorted(list(set(video_files))) # Sort and remove potential duplicates

        if not video_files:
            print(f"    Warning: No video files found in {video_folder_abs}")
            continue

        num_videos = len(video_files)
        print(f"    Found {num_videos} videos.")

        for idx, video_path in enumerate(video_files):
            video_filename = os.path.basename(video_path)
            feature_filename = os.path.splitext(video_filename)[0] + '.pkl'
            feature_save_path = os.path.join(feature_class_dir, feature_filename)

            # Check if features already exist
            if os.path.exists(feature_save_path):
                print(f"({idx+1}/{num_videos}) Skipping {video_filename}, features already exist.")
                skipped_files_count += 1
                continue

            print(f"    ({idx+1}/{num_videos}) Processing {video_filename}...")
            keypoints_data = extract_keypoints_from_video(video_path, FRAME_SKIP, SEQUENCE_LENGTH)

            if keypoints_data is not None:
                try:
                    with open(feature_save_path, 'wb') as f:
                        pickle.dump(keypoints_data, f)
                    print(f"Saved features to {feature_save_path}")
                    processed_files_count += 1
                except Exception as e:
                    print(f"      Error saving features for {video_filename}: {e}")
                    failed_files_count += 1
            else:
                print(f"Failed to extract features for {video_filename}. Skipping.")
                failed_files_count += 1

    print(f"\nFinished processing {data_directory}")
    print(f"  Successfully processed: {processed_files_count} files")
    print(f"  Skipped (already exist): {skipped_files_count} files")
    print(f"  Failed (extraction/saving): {failed_files_count} files")
    return processed_files_count, skipped_files_count, failed_files_count

print("Mediapipe Pose initialized and helper functions defined.")

## Cell 4: Feature Extraction Execution

This cell runs the main feature extraction process. It calls `process_data_folder` for both the `TRAIN_DATA_DIR` and `TEST_DATA_DIR` specified in Cell 2. This will populate the `FEATURES_OUTPUT_DIR` with `.pkl` files containing the extracted keypoint sequences for all videos found. It also times the process and reports the counts of processed, skipped, and failed files. Finally, it closes the Mediapipe `pose` object.

In [ ]:
# --- Main Preprocessing Execution ---
print("\nStarting Feature Extraction Process...")
overall_start_time = time.time()
total_processed = 0
total_skipped = 0
total_failed = 0

# Process Training Data Directory
if os.path.isdir(TRAIN_DATA_DIR):
    processed, skipped, failed = process_data_folder(
        TRAIN_DATA_DIR,
        CLASS_FOLDERS,
        label_map,
        FEATURES_OUTPUT_DIR
    )
    total_processed += processed
    total_skipped += skipped
    total_failed += failed
else:
    print(f"\nWarning: Train data directory not found: {TRAIN_DATA_DIR}")

# Process Testing Data Directory
if os.path.isdir(TEST_DATA_DIR):
    processed, skipped, failed = process_data_folder(
        TEST_DATA_DIR,
        CLASS_FOLDERS,
        label_map,
        FEATURES_OUTPUT_DIR
    )
    total_processed += processed
    total_skipped += skipped
    total_failed += failed
else:
    print(f"\nWarning: Test data directory not found: {TEST_DATA_DIR}")

overall_end_time = time.time()
print(f"\n--------------------------------------------------")
print(f"Overall Feature Extraction Summary:")
print(f"  Total time taken: {overall_end_time - overall_start_time:.2f} seconds.")
print(f"  Total files processed successfully: {total_processed}")
print(f"  Total files skipped (features existed): {total_skipped}")
print(f"  Total files failed (extraction/saving): {total_failed}")
print(f"  Features saved in subfolders under: {FEATURES_OUTPUT_DIR}")
print(f"--------------------------------------------------")

# Close the pose object when done
print("\nClosing Mediapipe Pose object...")
try:
    pose.close()
    print("Mediapipe Pose object closed.")
except Exception as e:
    print(f"Error closing pose object (might have already been closed or not initialized): {e}")

print("\n--- Feature Extraction Script Finished ---")